In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [30]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=9,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 648,964 || all params: 125,297,672 || trainable%: 0.5179


In [31]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [32]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.374500,1.343780,0.664062
200,1.305600,1.244221,0.828125
300,1.114700,0.932964,0.860938
400,0.713400,0.486278,0.879687
500,0.451600,0.348204,0.885938
600,0.388800,0.331520,0.887500
700,0.354900,0.322565,0.895312
800,0.374300,0.312316,0.900000
900,0.346900,0.314025,0.901563
1000,0.343400,0.310289,0.900000


In [34]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:56<00:00,  4.39it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [33]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3745,1.343780,0.330625,0.664062,0.000048,0.013405,0.030720,1.022861,-0.333437,0.497882
1,200,1.3056,1.244221,0.579018,0.828125,0.000045,0.026810,0.061379,1.049331,-0.249107,0.699191
2,300,1.1147,0.932964,0.805357,0.860938,0.000043,0.040214,0.181736,1.194794,-0.055580,0.935442
3,400,0.7134,0.486278,0.871429,0.879687,0.000040,0.053619,0.227122,1.467061,-0.008259,0.990612
4,500,0.4516,0.348204,0.883036,0.885938,0.000037,0.067024,0.103396,1.296940,-0.002902,0.996725
5,600,0.3888,0.331520,0.884821,0.887500,0.000035,0.080429,0.057280,1.172781,-0.002679,0.996982
6,700,0.3549,0.322565,0.887946,0.895312,0.000033,0.093834,0.032335,1.100242,-0.007366,0.991773
7,800,0.3743,0.312316,0.877232,0.900000,0.000030,0.107239,0.061984,1.198464,-0.022768,0.974702
8,900,0.3469,0.314025,0.892411,0.901563,0.000028,0.120643,0.032875,1.104688,-0.009152,0.989849
9,1000,0.3434,0.310289,0.888839,0.900000,0.000025,0.134048,0.033111,1.106711,-0.011161,0.987599
